<h1>Pseudocode for a minimax algorithm (Adapt this for our game)</h1>

<h1> Nim Rules (for reference) </h1>

<h2> To Dos </h2>

1) We already have a great minimax pseudocode--it's really not that complicated. We just need to make a tree for Nim and adapt the minimax algorithm as needed

    1b) Test the Tree!

<h1> Nim Rules & Tree Setup </h1>

In [144]:
import copy

In [145]:
class Stack:
    def __init__(self, size):
        self.size = size
    
    def removeStone(self, amount):
        if amount <= self.size and amount != 0:
            self.size -= amount
            return True #Indicates a valid turn
        
        return False #Indicates an invalid turn

In [146]:
nim = [Stack(3), Stack(3), Stack(3)]

In [147]:
for stack in nim: print(stack.size)

3
3
3


In [148]:
class Node:
    def __init__(self, state, children): #Might need parent...but at the same time, do we even care about the parent? Would we use it?
        self.state = state
        #self.parent = parent
        self.children = children

In [149]:
def children_populate(nim):
    children = []
    stack_index = 0
    for each in nim:
        for i in range(1, each.size+1):
            temp_nim = copy.deepcopy(nim)
            temp_nim[stack_index].removeStone(i)
            children.append(Node(temp_nim, children_populate(temp_nim)))
        stack_index += 1
    return children

#This no longer works per changing the children to Nodes instead of Stacks
test_children = children_populate(nim)
for stack in nim: print(stack.size)
for child in test_children:
    for stack in child: print(stack.size, end = " ")
    print("\n")

In [151]:
class NimTree:
    def __init__(self, nim):
        self.nim = nim
        self.origin = Node(nim, children_populate(nim)) #Theoretically, because children are recursively created in the children_populate function...this should work. Theoertically.

In [152]:
nim = [Stack(3), Stack(3), Stack(3)]
tree = NimTree(nim)

#Okay Marcus, I'll explain some of what I've done thus far.
#Nim is a game w/ a collection of an arbitrary number of stacks & an arbitrary number of stones per stack
#Players take turns removing stones (they can remove as many as they want / can (no negative stones) or as a few as they want / can but HAVE to remove 1)
#So what I've done is thus:
#The Stack class acts as a stack
#nim is a starting state of the game, represented by an array of stacks.
#Node is simply a node containing a current state of nim (an array of stacks) that also links to its children
#I didn't include parent because I don't think we need it and the way I populated the tree was a little recursive and wonky, and it didn't play well w/ parent. We can fix this pretty easily though.
#NimTree is a class containing, you guessed it, a tree of Nim Nodes. 
#Currently, I think it's working right. However, I haven't had time to test it. Do you think you can test and troubleshoot the tree? Should be able to do this with a for loop
#It should look like slide 14 of chapter 7 notes.
#You can use the following lines of code to print nicely

In [154]:
for stack in tree.origin.state: print(stack.size, end = " ")
print("\n") #this prints the nim state of the origin node of the tree

3 3 3 



#Outdated since children are Nodes.
for child in tree.origin.children:
    for stack in child: print(stack.size, end = " ")
    print("\n")

In [158]:
for child in tree.origin.children: #This prints the nim state of all of the children nodes of a node (in this case, origin)
    for stack in child.state: print(stack.size, end = " ")
    print("\n")

2 3 3 

1 3 3 

0 3 3 

3 2 3 

3 1 3 

3 0 3 

3 3 2 

3 3 1 

3 3 0 



TO DO FOR MARCUS
Psuedocode:
for each node in tree:
    verify proper children (if any) are being generated per slide 14 of chapter 7
    also test larger nim games (both stack # and stack size)
    
    Maybe start looking at incorporating the minimax algorith? Shouldn't be too hard w/ a tree in place
    
You'll do great :) Just get done what you can and we can keep working on it together.

Also, as a tip:
click Cell->run all when you start & periodicailly if you update code.
Shift + Enter runs one block
Cell->Cell Type->Markdown is good for writing notes like this.

<h1> MiniMax Tree </h1>

<h1> User Input & Test Playing </h1>